In [2]:
import numpy as np


import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import HTML

<h2><center>Artificial Intelligence, Assignment 2</center></h2>

<h3><center>Search agents</center></h3>

<img src='imageAIAssignment002.jpeg' width="400" height="400">


__Total:__ 27pts + 5pts

__Given date:__ Tuesday September 28

__Due date:__ Friday October 15

### Question 1. A story of robots and batteries 

##### (15pts + 2pts)

We consider the simple $12\times12$ world depicted below. In this first exercise, we will study the behavior of an agent that can only see the immediately adjacent cells (that is it only sees the cells that are directly in front, behind, or on its left/right). Your agent is a simple robot that enters the maze from the bottom left cell and must reach the exit which is located on the uppermost rightmost cell. 


<img src='MAZE002.png' width="400" height="400">

<p style="margin-bottom:1cm;"></p>

The objective of the agent is twofold:

   - 1) It has to find the exit (In a first approach, we won't take any step cost into account), while avoiding all the holes.

   - 2) It has to collect all the batteries.



##### Question 1.1. (5pts) A simple reflex agent 

Using a simple while loop and follow the ideas discussed during the recitations to code a simple reflex agent that achieves this objective. When the agent faces a pit, it should avoid it. When the agent is on a cell containing a battery, it should collect it. Finally the agent can only move in the four immediately adjacent cells to its current position. When it sees no pit and there are no batteries in any of the adjacent cells, it should move at random. Consider adding on the order of 14 batteries and 18 holes (first manually, then at random) 


##### Answer 1.1

The below is a simple impletations of this simple reflex agent.

In [58]:
# an aux function
def children(pos, avoids=[-1,-2]):
    ret = []
    x, y = pos
    for xx in [x-1,x+1]:
        if xx>= 0 and xx<shape[0] and (world[xx,y] not in avoids):
            ret.append((xx,y))
    for yy in [y-1,y+1]:
        if yy>= 0 and yy<shape[1] and (world[x,yy] not in avoids):
            ret.append((x,yy))
    return ret

In [57]:
def table_gen(shape=(12,12), num_bat=14,num_holes=18,num_walls=0 ,seed=1):
    np.random.seed(seed)
    size = shape[0]*shape[1]
    
    assert(size>5)
    
    world = np.zeros(size)
    
    # generating the batteries and holes randomly.
    i = 0
    while i < num_bat:
        pos = np.random.randint(1,size-1)
        if world[pos] == 0:
            world[pos] = 1
            i += 1
    i = 0
    while i < num_holes:
        pos = np.random.randint(1, size-1)
        # the initial/exit position cannot be a hole. 
        if world[pos] == 0:
            world[pos] = -1
            i += 1
    i = 0
    while i < num_walls:
        pos = np.random.randint(1, size-1)
        # the initial/exit position cannot be a hole. 
        if world[pos] == 0:
            world[pos] = -2
            i += 1

    # -1 means there is a hole, 1 means there is a batteries, -2 means there is a wall. 
    
    world = world.reshape(shape)
    holes = np.array(np.where(world==-1))
    batteries = np.array(np.where(world==1))
    walls = np.array(np.where(world==-2))
    
    return (world, batteries, holes, batteries, walls)

In [5]:
def simple_reflex_agent() -> object:
    # initial states
    # selecting a random seed here
    np.random.seed(5)
    uncollected = set([tuple(i) for i in batteries.transpose()])
    # this sets contains the uncollected batteries. 
    end_pos = (shape[0]-1,shape[1]-1)
    curr_pos = (0, 0)
    
    while (len(uncollected) > 0) or (curr_pos != end_pos):
        # update the agent's position
        if world[curr_pos] == 1 and curr_pos in uncollected:
            uncollected.discard(curr_pos)
        next_positions = children(curr_pos)
        for next_pos in next_positions:
            if world[next_pos] == 1 and next_pos in uncollected:
                curr_pos = next_pos
                continue
        curr_pos = next_positions[np.random.choice(len(next_positions))]
        
        yield (curr_pos, uncollected)


## animation of simple reflex agent

In [6]:
shape = (12,12)
num_bat = 14
num_holes = 18
num_walls = 0

world, batteries, holes, battery, walls = table_gen(shape=(12,12), num_bat=14,num_holes=18,num_walls=0 ,seed=1)

In [ ]:
%matplotlib inline

#### plotting, before running the agents
fig, ax = plt.subplots()
ax.matshow(np.zeros(shape=world.shape), cmap=plt.cm.BuGn)

# the agent
agent, = plt.plot([],[],marker='$😀$',ms=10,linestyle='',color='tan')

# the holes and batterys
plt.plot(holes[0], holes[1], marker='$⚪$', ms=10,color='black',linestyle='')
uncol_batt, = plt.plot([], [], marker='$🔋$',ms=10, linestyle='')

# grid
ax.set_xticks(np.arange(world.shape[0])-0.5,minor='true')
ax.set_yticks(np.arange(world.shape[1])-0.5, minor='true')
ax.grid(which='minor')
ax.set_xticks([])
ax.set_xticklabels([]) 
ax.set_yticks([]) 
ax.set_yticklabels([])
plt.xlim((-0.5,world.shape[0]-0.5))
plt.ylim((-0.5,world.shape[1]-0.5))
plt.grid()


def update_anime(frame):
    curr_pos, uncollected = frame
    agent.set_data(curr_pos[0],curr_pos[1])
    if len(uncollected):
        un_x, un_y = np.array(list(uncollected)).transpose()
        uncol_batt.set_data(un_x,un_y)
    else:
        uncol_batt.set_data([],[])
    return (agent,uncol_batt)
    
ani = animation.FuncAnimation(fig, update_anime, interval=100,frames=simple_reflex_agent(),blit=True,save_count=len([1 for i in simple_reflex_agent()]))
ani.save('simple reflex animation.gif')

See the animation here. (Notice that since I have selected random seed, when you run it again, you will get the same animation. So probably you set a different seed to get different animations)

<img src="simple reflex animation.gif" width="750" align="center">

##### Question 1.2. (5pts) Search agent

We will now assume that our agent has a map of the world. On top of the pits from above, the world now also contains walls, which are additional obstacles in the search for the exit.

Solve the problem using Breadth First search. The children of a node are given by the adjacent cells. Once you have the path to a battery, stores it. Then continue BFS from the location of this battery and store your second path,.... Proceed like this until you have all the batteries. From the last battery find the exit.  

<img src='Maze003.png' width="400" height="400">

<p style="margin-bottom:1cm;"></p>



##### Anwser 1.2 

In [60]:
shape = (18,18)
num_bat = 20
num_holes = 18
num_walls = 18

world, batteries, holes, battery,walls = table_gen(shape=shape, num_bat=num_bat,num_holes=num_holes,num_walls=num_walls,seed=3)

In [19]:
def minimal_path(start, end=end_pos):
    '''
    this returns a path from curr_pos to the exit.
    '''
    if start == end:
        return []
    
    explored = [start]
    q = [start]
    bp = {}
    bp[start] = None
    
    while (end not in explored) and (len(q) > 0):
        new_pos = []
        for pos in q:
            for child in children(pos):
                if child not in explored:
                    explored.append(child)
                    new_pos.append(child)
                    bp[child] = pos
        q = new_pos
    
    if end in explored:
        ret = []
        curr_pos = end
        while bp[curr_pos] != None:
            ret.append(curr_pos)
            curr_pos = bp[curr_pos]
        ret.reverse()
        return ret
    else:
        return None

In [64]:
def find_next_bat(path):
    collected = set([i for i in path if world[i]==1])
    start = path[-1]
    
    if len(collected) == num_bat:
        npath = minimal_path(start,end_pos)
        if npath!=None:
            return path + npath
        else: return None

    explored = [start]
    q = [start]
    bp = {}
    bp[start] = None
    found = False
    
    while (not found) and (len(q) > 0):
        new_pos = []
        for pos in q:
            for child in children(pos):
                if child not in explored:
                    explored.append(child)
                    new_pos.append(child)
                    bp[child] = pos
                    if world[child] == 1 and (child not in collected):
                        found = True
                        batt = child
        q = new_pos
    
    if found:
        npath = []
        curr_pos = batt
        while bp[curr_pos] != None:
            npath.append(curr_pos)
            curr_pos = bp[curr_pos]
        npath.reverse()
        
        return path + npath
    else: 
        return None

In [71]:
def search_agent() -> object:
    # initial states
    collected = set()
    # this sets contains the collected batteries. 
    end_pos = (shape[0]-1,shape[1]-1)
    start_pos = (0, 0)
    
    path = [start_pos]
    
    while path[-1] != end_pos:
        path = find_next_bat(path)
        yield path

In [ ]:
%matplotlib inline

#### plotting, before running the agents
fig, ax = plt.subplots()
ax.matshow(np.zeros(shape=world.shape), cmap=plt.cm.BuGn)

# the agent
agent, = plt.plot([],[],marker='$😀$',ms=10,linestyle='',color='tan')

# the holes and batterys
plt.plot(holes[0], holes[1], marker='$⚪$', ms=10,color='black',linestyle='')
plt.plot(walls[0], walls[1], marker='$🔴$', ms=10,color='black',linestyle='')
uncol_batt, = plt.plot([], [], marker='$🔋$',ms=10, linestyle='')

# grid
ax.set_xticks(np.arange(world.shape[0])-0.5,minor='true')
ax.set_yticks(np.arange(world.shape[1])-0.5, minor='true')
ax.grid(which='minor')
ax.set_xticks([])
ax.set_xticklabels([]) 
ax.set_yticks([]) 
ax.set_yticklabels([])
plt.xlim((-0.5,world.shape[0]-0.5))
plt.ylim((-0.5,world.shape[1]-0.5))
plt.grid()

i=0
def update_anime(frame):
    x,y = np.array(frame).transpose()
    agent.set_data(x,y)
    uncollected = [tuple(i) for i in batteries.transpose() if tuple(i) not in frame]
    if len(uncollected):
        un_x, un_y = np.array(list(uncollected)).transpose()
        uncol_batt.set_data(un_x,un_y)
    else:
        uncol_batt.set_data([],[])
    return (agent,uncol_batt)
    
ani = animation.FuncAnimation(fig, update_anime, interval=1000,frames=search_agent(),blit=True)
ani.save('first search reflex animation.gif')

See the animation here. (Notice that since I have selected random seed, when you run it again, you will get the same animation. So probably you set a different seed to get different animations)

<img src="first search reflex animation.gif" width="750" align="center">

##### Question 1.3. (5pts) Informed search agent

In this third question, we will use an informed search strategy to improve our agent. We want to use as our heuristic the $\ell_1$ distance to the closest battery that has not been picked. Code a Best First Search agent whose heuristic changes as it picks up new batteries. As soon as it picked up the last battery, the heuristic becomes the $\ell_1$ distance to the exit. You can assume that the cells have unitary side length. Also recall that the $\ell_1$ distance is given by $\|\boldsymbol x_1 - \boldsymbol x_2\|_1 = |x_{11} - x_{21}| + |x_{12} - x_{22}|$ where $\boldsymbol x_1 = (x_{11}, x_{12})$, $\boldsymbol x_{2} = (x_{21}, x_{22})$.    


<img src='Maze003.png' width="400" height="400">

<p style="margin-bottom:1cm;"></p>



In [12]:
# generating the table

shape = (12,12)
size = shape[0]*shape[1]

num_bat = 14
num_holes = 14
num_wall = 8

world = np.zeros(size)

# it should not be 0, which would be the starting position. 

# generating the batteries, holes and walls randomly. 
# 0 means there is nothing
# 1 means there is a battery
# -1 means there is a wall or a hole.

i = 0
while i < num_bat:
    pos = np.random.randint(0,size)
    if world[pos] == 0:
        world[pos] = 1
        i += 1

i = 0
while i < num_holes:
    pos = np.random.randint(1, size-1)
    # the randint starts from 1 because the initial position cannot be a hole. 
    if world[pos] == 0:
        world[pos] = -1
        i += 1
i = 0
while i < num_bat:
    pos = np.random.randint(0,size-1)
    if world[pos] == 0:
        world[pos] = 2
        i += 1

world = world.reshape(shape)
holes = np.where(world==-1)
batteries = np.where(world==1)
walls = np.where(world==2)

world[world==2] = -1
# this merges the walls and holes as the same thing. 

In [13]:
# initial states
collected = set()
# this sets contains the collected batteries. 
end_pos = (shape[0]-1,shape[1]-1)
start_pos = (0, 0)

bat_coor = set(zip(batteries[0],batteries[1]))

path = [start_pos]

In [14]:
def informed_search(path):
    collected = set([i for i in path if world[i]==1])
    start = path[-1]
    
    if len(collected) == num_bat:
        npath = minimal_path(start,end_pos)
        if npath != None:
            return path + npath
        return None
    
    mindist = size*size
    nextbat = None
    
    for bat in bat_coor:
        if bat not in collected:
            d = abs(bat[0] - start[0]) + abs(bat[1] - start[1])
            if d < mindist:
                mindist = d
                nextbat = bat
    
    assert(nextbat!=None)
    
    npath = minimal_path(start, nextbat)
    if npath != None:
        return informed_search(path+npath)
    else:
        return None

In [15]:
'   '.join([str(i) for i in informed_search([(0,0)])])

'(0, 0)   (1, 0)   (1, 1)   (2, 1)   (2, 2)   (3, 2)   (2, 2)   (2, 3)   (2, 4)   (2, 5)   (3, 5)   (2, 5)   (1, 5)   (0, 5)   (1, 5)   (2, 5)   (3, 5)   (4, 5)   (4, 6)   (4, 7)   (4, 8)   (3, 8)   (3, 9)   (3, 10)   (2, 10)   (2, 11)   (1, 11)   (0, 11)   (1, 11)   (2, 11)   (3, 11)   (4, 11)   (4, 10)   (5, 10)   (6, 10)   (7, 10)   (8, 10)   (8, 11)   (9, 11)   (9, 10)   (9, 9)   (9, 8)   (9, 7)   (9, 6)   (9, 5)   (9, 4)   (10, 4)   (10, 3)   (11, 3)   (10, 3)   (10, 2)   (10, 1)   (10, 0)   (9, 0)   (8, 0)   (9, 0)   (10, 0)   (10, 1)   (10, 2)   (9, 2)   (9, 3)   (9, 4)   (9, 5)   (9, 6)   (10, 6)   (11, 6)   (11, 7)   (11, 8)   (11, 9)   (11, 10)   (11, 11)'

##### Bonus (2pts) 
Generate and display the movie of the search for each of the questions above. 

I have displayed the movies for the simple reflex agent as well as the bfs agent. However, I don't really have the time to make a movie for the informed search agent again



### Question 2. Rook Jumping

##### (12pts + 3pts)

In this second question, we consider a "rook jumping" maze. An example of such a maze is given below (the starting position is shown in red and the goal position is shown in green). 



<img src='RookMaze001.png' width="350" height="350">

Each state in the maze has an associated jump number that provides the exact number of cells one may move horizontally or vertically in a straight line to change state. As an example, in the maze above, the first move may either be 2 cells on the right of (0,0) or 2 cells down to (2,0).

#### Question 2.1. Generate the maze (2pts)

Start by completing the function Maze_generation which takes as argument the dimension of the maze as well as a maximum jump number (don't take it much larger than n/2) and returns a matrix of random integers between 0 and the maximum jump length. 

In [79]:
def Maze_generation(n, m=None):
    '''function should return a random maze'''
    if m:
        return np.random.randint(low=1, high=m, size=(n,n))
    else:
        return np.random.randint(low=1, high=n//2, size=(n,n))

In [80]:
def display_maze(maze):
    fig, ax = plt.subplots()
    max_val = max(maze.shape)    
    ax.matshow(maze, cmap=plt.cm.Blues)

    for i in range(max_val):
        for j in range(max_val):
            c = maze[j,i]
            ax.text(i, j, str(c), va='center', ha='center')


    ax.set_xticks([]) 
    ax.set_xticklabels([]) 
    ax.set_yticks([]) 
    ax.set_yticklabels([]) 
    plt.show()

##### Question 2.2. (5pts) Maze Evaluation

We now want to solve the maze (or equivalently, make sure it has a solution)  

Using _Breadth First Search_ (start with a reasonably small maze, e.g. 5 by 5), compute the minimum distance (depth number of moves) to each cell from the start cell (take the start cell to be the _uppermost leftmost_ cell). For this, keep track of a list of the depth distances to each node and update this list each time you encounter the corresponding node. Once BFS completed, return the minimum element from the list. 

Finally return the negative of the minimmum length of the path from the start to the goal and a large positive number (e.g. 1e6) if there is no such path.


In [81]:
def Maze_Evaluation(maze):
    
    '''The function takes a maze (random n x n array of integers)
    and should return the the minimum 
    distances of the start node to the goal'''
    d = maze.shape(0)
    dis = np.zeros(maze.shape) + np.inf
    start = (0,0)
    dis[start] = 0
    
    q = [start]

    def children(pos):
        x,y = pos
        steps = maze[pos]
        ret = []
        for xx in [x - steps, x + steps]:
            if xx>= 0 and xx < d:
                ret.apped((xx,y))
        for yy in [y - steps, y + steps]:
            if yy>= 0 and yy < d:
                ret.apped((x,yy))
        return ret

    
    while len(q):
        newq = []
        for pos in q:
            assert(np.isfinite(dis[pos]))
            for child in children(pos):
                if not np.isfinite(dis[child]):
                    dis[child] = dis[pos] + 1
                    newq.append(child)
        q = newq
    
    if np.isfinite(dis[(-1,-1)]):
        return - abs(dis[(-1,-1)])
    else: return np.inf

##### Question 2.3. (5pts) Stochastic local search (Hill climbing)

Now that we have a way of representing each maze, we will try to improve our maze with a stochastic local search. In this question, each state in our graph will encode a whole maze. Our local search algorithm will work as follows

- For a random, non goal cell, change the jump number to a different random legal jump number

- Re-evaluate the start to goal distance according to the _Maze_Evaluation_ function that you implemented in Question 2.2.

- If the objective function has not increased, accept the change and store the new maze if its evaluation is the best evaluated so far. Otherwise, reject the change and revert the cell to its previous jump number

Perform a few iterations of Stochastich HC and return the RJM with the best (minimum) objective function

In [82]:
def Maze_improvement(maze_init, maxIter):
    
    '''The function takes an initial Rook Jumping maze and 
    a maximum number of iterations as an input and returns 
    the improvement of the original maze obtained 
    through maxIter iteration of Stochastic Hill Descent'''
    
    maze = maze_init.copy()
    steps = Maze_Evaluation(maze)
    d = maze.shape[0]
    
    non_goal_cells = [(i//d,i%d) for i in range(d*d-1)]
    
    for i in range(maxIter):
        non_goal_cell = non_goal_cells[np.random.choice(len(non_goal_cells))]
        new_val = np.random.randint(1,d//2)
        old_val = maze[non_goal_cell]
        
        maze[non_goal_cell] == new_val
        score = Maze_Evaluation(maze)
        if score < steps:
            steps = score
        else:
            maze[non_goal_cell] = old_val
    
    return maze
    

##### Bonus (3pts) Random Restart

One problem with pure hill descent is that stochastic local search may become trapped in local minima. A possible escape strategy is to restart the search periodically. Another way of viewing this is that we iteratively perform pure hill descent, starting each descent at a random state. The end result is the best result from all descents.

Add Random Restart to your 'Maze_improvement' function.

In [83]:
def Maze_improvement_RR(maze_init, max_iter_SHD, max_iter_RR):
    
    '''The function takes an initial (random) Rook Jumping maze and 
    a maximum number of iterations as an input and evaluates 
    the improvement of the original maze obtained 
    through max_iter_SHD iterations of Stochastic Hill Descent. 
    This process is repeated max_iter_RR times 
    (with different random initializations) and each of the 
    obtained solutions are stored. The max_iter_RR solutions are finally compared in 
    terms of the cost between the start and the goal node and 
    the best solution is returned as the final output'''
    
    ret = []
    
    for i in range(max_iter_RR):
        
        new_maze = Maze_improvement(maze_init,max_iter_SHD)
        new_value = Maze_Evaluation(new_maze)
        
        ret.append(new_maze, new_value)
        
    ret.sort(key=lambda x:x[1])
    
    return ret[0]